In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import re
from selenium.common.exceptions import NoSuchElementException

def extract_job_title_from_result(soup): 
    jobs = []
    for div in soup.find_all(name="div", attrs={"class":"row"}):
        for a in div.find_all(name="a", attrs={"data-tn-element":"jobTitle"}):
            jobs.append(a["title"])
            
    return(jobs)

def access_extract_data(company):
    # create a new Firefox session
    url = "https://www.indeed.com/q-{}-jobs.html".format(company)
    driver = webdriver.Firefox(executable_path = '/Users/aniketmuluk/Downloads/geckodriver-1')
    driver.implicitly_wait(30)
    driver.get(url)
    try:
        dismiss_button = driver.find_element_by_id("searchCountPages")
        dismiss_button.click()
    except NoSuchElementException:
        pass
        
    try:
        nu_pages = driver.find_element_by_id("searchCountPages")
        number_pages = nu_pages.text
        number_pages = number_pages.replace(",","")
        Number_of_pages = (map(int, re.findall(r'\d+', number_pages))[1])//10
        n = Number_of_pages+1
    except NoSuchElementException:
        return None 
    
    List_job_titles =[]
    i=2
    for i in range(2,n+1):
        html = driver.find_element_by_id('resultsCol')
        data_html = html.get_attribute('innerHTML')
        soup = BeautifulSoup(data_html, 'html.parser')
        List_job_titles.append(extract_job_title_from_result(soup))
        try:
            python_button = driver.find_element_by_link_text('{}'.format(i))
            python_button.click()
            time.sleep(10)
            driver.refresh()
        except:
            break
    
        
    return List_job_titles

def main():
    company = str(raw_input("Enter Company Name:"))
    company = company.replace(" ","+")
    job_titles = access_extract_data(company)
    if job_titles == None:
        print "No Jobs"
    else:
        flat_list_job_titles = []
        for sublist in job_titles:
            for item in sublist:
                flat_list_job_titles.append(item)
        df = pd.DataFrame(flat_list_job_titles)
        df.to_csv('{}.csv'.format(company), sep='\t', encoding='utf-8')

if __name__ == "__main__":
    main()

Enter Company Name:MILLENIUM
